In [1]:
import ee
import geemap
import geopandas as gpd
from ipyleaflet import GeoJSON
from zipfile import ZipFile
ee.Initialize()

# Documentation = https://stackoverflow.com/questions/66374322/imagecollection-filterbounds-is-not-showing-results-from-geometry-input
# https://www.marineregions.org/downloads.php #Se descarga el archivo en Marine and land zones: the union of world 
#country boundaries and EEZ's, version 3 preferiblemente

Map = geemap.Map()

# retreives geometry for Colombia
Colombia = ee.FeatureCollection('users/hmojica/data/departamentos')
departamentos = Colombia.filterMetadata('DPTO_CNMBR', 'equals', 'VICHADA')


# Para filtrar por departamento: .filterMetadata('DPTO_CNMBR', 'equals', 'VICHADA')

style = {'color': '000000', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
Map.addLayer(Colombia.style(**style), {}, "Departamentos")

#Map.addLayer(departamentos, {}, "Departamentos2")


# clips image to colorado geometry

s5p_methane_dataset = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4").filterBounds(departamentos)


# selects dataset to be mapped

s5p_metane = s5p_methane_dataset.select('CH4_column_volume_mixing_ratio_dry_air_bias_corrected').filterDate('2022-01-01', '2023-01-01')


# Clip to bounds of geometry

s5p_metane_img = s5p_metane.map(lambda image: image.clip(departamentos))

# sets image variables

s5p_methane_viz = {
    'min': 1750,
    'max': 1900,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'opacity': 1.0
}

# adds image layers to map
Map.addLayer(s5p_metane_img, s5p_methane_viz, 'methane')
Map.centerObject(Colombia, 5);
Map


Map(center=[3.8923553006509, -73.07874943313837], controls=(WidgetControl(options=['position', 'transparent_bg…

In [ ]:
#Código en Java para consol de gee
# Para cargar el archivo en GEE https://www.youtube.com/watch?v=AmaaEFbBtfQ

# var shp=ee.FeatureCollection(polygon);
# var Front_col=shp.filter(ee.Filter.eq('UNION', 'Colombia'));
# var Front_col_s = Front_col.style({fillColor : 'b5ffb4',
#   width: 2})
# Map.addLayer(Front_col_s,{},'Frontera',1,0.2)
# var methane = "COPERNICUS/S5P/OFFL/L3_CH4"
# //var Municipios= ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")#Datos de fronteras a nivel municipal
# //Dep = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")#Filto de los anteriores datos para Colombia
# //Dep_col=Dep.filter(ee.Filter.eq('ADM0_NAME', 'Colombia'))#Datos de fronteras a nivel departamental
# //Mun_col=Municipios.filter(ee.Filter.eq('ADM0_NAME', 'Colombia'))#Filtro de los anteriores datos para Colombia
# //Fronteras=ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")#Datos de fronteras nacionales
# //Front_col=Fronteras.filter(ee.Filter.eq('ADM0_NAME', 'Colombia'))#Filtro de los anteriores datos para Colombia
# var s_date='2022-08-09' 
# var e_date='2023-02-09'
# var image1 = ee.ImageCollection(methane).select("CH4_column_volume_mixing_ratio_dry_air_bias_corrected").filterDate(s_date,e_date)
# var imageC=ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CLOUD").select('cloud_fraction').filterDate(s_date,e_date)

# var imageNO2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2').select('tropospheric_NO2_column_number_density').filterDate(s_date,e_date);

# var imageSO2=ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_SO2").select('SO2_column_number_density').filterDate(s_date,e_date);


# // Map.setCenter(65.27, 24.11, 4);
# //img_col= image1.first()
# var img1 = image1.mean()
# var img_col=img1.clip(Front_col)
# var imageC_col=imageC.mean()
# var img_col2=imageC_col.clip(Front_col)
# //img_col = image1.first()
# var img_col_NO2=imageNO2.mean().clip(Front_col)
# var img_col_SO2=imageSO2.mean().clip(Front_col)
# var band_viz = {
#     'min': 1850,
#     'max': 2000,
#     'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
# }

    
# //Map = geemap.Map(center=(4.6,-74),zoom=5)
# //ap.setCenter(-74.297333, 4.570868,5)
# //Map = geemap.Map(center=(-23, -48.4), zoom=9)

# //Map.setCenter(-74.297333, 4.570868,5)
# //imageNO2.getInfo()
# Map.addLayer(img_col,band_viz,'Methane')
# Map.addLayer(img_col2,{'min': 0,'max': 1,
#     'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
# },'Clouds')
# Map.addLayer(img_col_SO2,{
#   'min': 0,
#   'max': 0.0005,
#   'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']},'SO2',1,0.7)
# Map.addLayer(img_col_NO2,{
#   'min': 0,
#   'max': 0.00005,
#   'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}, 'S5P N02',1,0.9)
# Map